<a href="https://colab.research.google.com/github/ghluque/Analisis_BTC/blob/master/ToolDash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#INFO: Como ejecutar una app Dash en Google Colab

Spoiler: le usamos la VM como si fuera un linux cualquiera :D


In [0]:
#S: Install Dash
!pip install dash
!pip install dash-html-components
!pip install dash-core-components
!pip install dash-table

     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 1.1MB 14.4MB/s 
     |████████████████████████████████| 3.3MB 51.1MB/s 
     |████████████████████████████████| 194kB 56.7MB/s 
     |████████████████████████████████| 1.8MB 26.9MB/s 
  Created wheel for dash: filename=dash-1.9.1-cp36-none-any.whl size=71512 sha256=02544b75fe4306644664861fad9addbe0c20dbc99b28ab2138cfc630327b0856
  Stored in directory: /root/.cache/pip/wheels/ae/29/73/dd1e3699c9b9e5488b9688fed5d990364317c8472004f8df0b
  Created wheel for flask-compress: filename=Flask_Compress-1.4.0-cp36-none-any.whl size=3712 sha256=6642a4bd3cb9f8cbf5c8cffb5e3fb7403a157b7c2c3f55e451034ef9452cc0c1
  Stored in directory: /root/.cache/pip/wheels/96/32/88/a1f6d9dd3c29570ab3a8acc0d556b3b20abcf3c623c868ce0a
  Created wheel for dash-renderer: filename=dash_renderer-1.2.4-cp36-none-any.whl size=1146887 sha256=4a4a09124c816e36e60b08a56860cbf44feb758794202edff4dd85013865b20c
  Stored in directory: /root

In [0]:
### Requerido1: ngrok para abrir un tunel SALIENDO desde la VM de Colab
# porque sino seria inaccesible desde tu navegador
# colab te da una maquina virtual que no tiene una IP publica
# ngrok abre un tunel SALIENDO desde la virtual a su sevidor
# y te permite conectarte al servidor dash que vas a ejecutar en la VM
# Se podria inventar algo parecido con ssh y un hosting, por ej.
import os.path;
import sys, json;
import requests;

import subprocess
Tunnel= None
TunnelUrl= None
def tunnel_start():
  global Tunnel, TunnelUrl
  if not os.path.isfile('ngrok'):
    !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
    !unzip -o ngrok-stable-linux-amd64.zip
  #A: si no teniamos ngrok lo bajamos

  if Tunnel and Tunnel.poll()!=None: #A: estaba activo pero termino
    Tunnel= None

  if not Tunnel: #A: no estaba ejecutando
    Tunnel= subprocess.Popen(["./ngrok","http","8050"]) #A: lanzamos el proceso
  
  tunnel_res= requests.get("http://localhost:4040/api/tunnels")
  tunnel_str= tunnel_res.text
  tunnel_cfg= json.loads(tunnel_str)

  TunnelUrl= tunnel_cfg['tunnels'][0]['public_url']
  print("Podes acceder desde "+TunnelUrl)
  return TunnelUrl  

In [0]:
#S: la aplicacion Dash
#A: Guardar un archivo en la VM de Colab, simplemente
#U: se podria bajar de github con !git clone o !wget

%%writefile my_dash_app1.py
#FROM: https://dash.plot.ly/interactive-graphing
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

available_indicators = df['Indicator Name'].unique()

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    return {
        'data': [dict(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }


def create_time_series(dff, axis_type, title):
    return {
        'data': [dict(
            x=dff['Year'],
            y=dff['Value'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }


@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


if __name__ == '__main__':
    app.run_server(debug=True)

Writing my_dash_app1.py


In [0]:
#S: ejecutar la app dash
tunnel_start()
!python my_dash_app1.py #A: hay que ejecutarla desde python como una app en la VM, no se puede desde este cuaderno

Podes acceder desde https://4a3be6a6.ngrok.io
Running on http://127.0.0.1:8050/
Debugger PIN: 609-526-431
 * Serving Flask app "my_dash_app1" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Running on http://127.0.0.1:8050/
Debugger PIN: 468-259-284


In [0]:
!node -e 'console.log("pepe")'



pepe
